### 读取文件

In [1]:
import numpy as np
import pandas as pd
import copy
import math
data_original_population = pd.read_csv('Population_original_each_cell.csv')
#population=data.values

datas=np.array(data_original_population) #读出来会多一列，因此需要删掉第一列，或者是从第二列开始读

original_p = datas[:,1:]
original_population = original_p
#print('the information for each cell：',original_population)
print('the height(长) of original_population：%s' % len(original_population) )
print('the height(宽) of original_population：%s' % len(original_population[0]) )

#the initial human is equal the population of that place


population_human=[]
population_zombie=np.zeros((234, 322))
#population_human=copy.deepcopy(original_population)
population_human=original_population.copy()


print("original_population[0][0] is :",original_population[0][0])
population_human[0][0]=1
print("population_human[0][0] is :",population_human[0][0])
original_population[0][0]

the height(长) of original_population：234
the height(宽) of original_population：322
original_population[0][0] is : 0
population_human[0][0] is : 1


0

In [2]:
import pandas as pd
data_elevation = pd.read_csv('Elevation_original_each_cell.csv')
#population=data.values
datas_correct=np.array(data_elevation) #读出来会多一列，因此需要删掉第一列，或者是从第二列开始读
elevation = datas_correct[:,1:]

In [3]:
#we will create a dict which has the information of Original_population,elevation,population_zombie,population_human
all_information_in_each_cell={}
i=0
j=0
print("the length of row is :",len(original_population))
print("the length of column is :",len(original_population[0]))

for i in range(len(original_population)):
    for j in range(len(original_population[i])):
        Original_population=original_population[i][j]
        Population_zombie=population_zombie[i][j]
        Population_human=population_human[i][j]
        Elevation = elevation[i][j]
        key=(i,j)
        all_information_in_each_cell[key]=[Original_population,Elevation,Population_zombie,Population_human]

        
#all_information_in_each_cell  #print all info

the length of row is : 234
the length of column is : 322


### 初始化信息

In [4]:
"""
initialize all the dictionaries
"""

zombies_dict = {}# 储存每个cell中已存活的天数的僵尸
zombie_human = {} # 储存每个cell将要计算的僵尸数和人口数

live_amount_zombies = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #每一位上对应的是存活的天数的数量 eg：live_amount_zombies[0]:新出生的僵尸数量
for x in range(234):
    for y in range(322):
       
        zombies_dict[(x,y)] = live_amount_zombies.copy()
        zombie_human[(x,y)] = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0],0]


In [5]:
# 初始化 僵尸病毒爆发当天
day = 1

#population_human=original_population#cell内人类数=原先该地区的人口数
#将【140，295】代表的rize地区初始化，人类数（human）归0，僵尸数=人口总数
population_zombie[140][295] = original_population[140,295].copy()
population_human[140][295] =0
zombie_human[(140,295)][0][0] = original_population[140,295]
zombie_human[(140,295)][1] =0


zombie_human[(140,295)]###同疑问；将sallow copy change to deep copy;then when we change the element of one list ,the other llst wont change

[[219433, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0]

In [6]:
all_information_in_each_cell[(140,295)] = [219433, 0.0, 219433, 0]
all_information_in_each_cell[(140,295)]

[219433, 0.0, 219433, 0]

In [7]:
zombies_dict[(140,295)][0]=219433

### 僵尸移动需要调用的函数
firstly,finding the neighbors surrounding the center point
secondly,calculating the population in each nearby cell and the elevation of the neighbors


In [8]:
def surrounds(center_point):
    """
    This function is to find the information of neighbors
    input should be a certain position of point p[x][y]=>p(x,y)
    outopt is a list of elements which including the position of all the neighbors
    """
    surround={}
    x=center_point[0]   
    y=center_point[1] 
    temporay=[[x-1,y-1],[x-1,y],[x-1,y+1],[x,y-1],[x,y+1],[x+1,y-1],[x+1,y],[x+1,y+1]]
    for i in range(len(temporay)):
        t=temporay[i]
        a=t[0]
        b=t[1]
        if (a >= 234 or b >=322 or a <= 0 or b <=0):
            all_information_in_each_cell[(a,b)] = [0,0,0,0]
            n = all_information_in_each_cell[(a,b)]
        else:
            n=all_information_in_each_cell[(a,b)]        
        surround[(a,b)]=n
    return surround


In [9]:
surrounds((140,295))

{(139, 294): [0, 0.0, 0.0, 0],
 (139, 295): [0, 0.0, 0.0, 0],
 (139, 296): [84644, 0.0, 0.0, 84644],
 (140, 294): [190816, 0.0, 0.0, 190816],
 (140, 296): [123140, 0.0, 0.0, 123140],
 (141, 294): [244828, 0.0, 0.0, 244828],
 (141, 295): [84046, 91.06, 0.0, 84046],
 (141, 296): [19605, 506.27, 0.0, 19605]}

In [10]:
def the_total_human_amount_of_surround(center_point):
    """
    This function calculates the total population of all neighbors     获取邻居的总人数
    the input is a point          输入中心内点的坐标
    output is an int       输出一个值为邻居里的总人数；为了算僵尸移动到别的区域做准备
    """
    neighbors=surrounds(center_point) 
    H = 0
    for i in neighbors.keys():
        information_of_neighbor = neighbors[i]
        human_amonut_of_neighbor = information_of_neighbor[3]
        H= human_amonut_of_neighbor + H  
    return H


In [12]:
print(the_total_human_amount_of_surround((140,295)))
print(read_information_elevation((140,295)))

747079
{(139, 294): 0.0, (139, 295): 0.0, (139, 296): 0.0, (140, 294): 0.0, (140, 296): 0.0, (141, 294): 0.0, (141, 295): 91.06, (141, 296): 506.27}


In [11]:
def read_information_elevation(center_point):
    """
    This function is to read the information of the elecation 读取海拔高度，中心点周边八个邻居的海拔高度
    返回所有邻居的海拔高度，c1，c2,c3,c4,c5,c6,c7,c8
    """
    neighbors=surrounds(center_point)
    H = 0
    elevation_of_neighbors={} # a dict to store the coordinates and the elevation of each neighbor
    for i in neighbors.keys():
        information_of_neighbor = neighbors[i]
        elevation_for_each_neighbor= information_of_neighbor[1]
        elevation_of_neighbors[i]=elevation_for_each_neighbor
    return elevation_of_neighbors

In [46]:
# if human amount arrounded the cell isn't 0, so zombies go to cell_n by percentage
def formule_zombies_to_cell_n (cell_start, cell_destination, lamda, human_total_surround, amount_human_destination, amount_zombies_start):#cell_m 是中心点的位置，cell_n 是邻居点
    """
    僵尸移动到邻近区域的数量
    cell_start : the center point(start point)
    cell_destination : 僵尸将要移动到的cell
    human_total_surround ： 中心点邻居们的总人数
    amount_human_destination ： 
    """
    percentage_human = amount_human_destination/human_total_surround
    cell_n_zombies_j_1 = percentage_human*amount_zombies_start*lamda
    
    return cell_n_zombies_j_1
    
# if total cell's human around egal 0, so zombies don't move, until die     
def formule_zombies_stop (cell_m, cell_n):   
    return 0

# if cell around isn't 0, so cell's zombies to 0 
# (conflict if total zombies_to_cell little than total cell's zombies) 
def formule_cell_zombies_empty(cell_m, cell_n):
    return 0


In [47]:
def calculate_lamda(center_point):#inputs is the elevation of the start cell 可以输入一列邻居的海拔高度，一个个的算 

    """
    input is the coordinates of the center point
    reutrn a dictionary with the position as key and the lamda between start position and this position
    """

    max_alpha = (math.pi)/18 #弧度 pi/18 = 10degree
    lamda_surround={}
    
    height_cell_m = all_information_in_each_cell[center_point][1]
    elevation_of_neighbors = read_information_elevation(center_point)
#    print("surround_elevation",surround_elevation)
    for i, value in elevation_of_neighbors.items():
#        print("surround",i)
        height=elevation_of_neighbors[i]
#        print("height",height) 
        if height_cell_m > height :
            difference = height_cell_m - height
        else:
            difference = height - height_cell_m
    
        tan = difference/15000
        alpha = math.atan(tan) #tan的反函数 return: 弧度 
    
        if alpha <= max_alpha:
            lamda = 1 - (0.1)*alpha
        else: 
            lamda = 0
        lamda_surround[i]=lamda
#        print(i, lamda)
        
    return lamda_surround

### 僵尸移动步骤

In [48]:
def zombies_evolution(all_information_in_each_cell, zombies_dict):
    surviving_zombies_with_age = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
    add_surviving_info_temp_dict = {}#{[]}
    del_surviving_info_temp_dict = {}#{[]}
    
    for x in range(234):
        for y in range(322):
            add_surviving_info_temp_dict[(x,y)] = surviving_zombies_with_age.copy()
            del_surviving_info_temp_dict[(x,y)] = surviving_zombies_with_age.copy()
    

    """
    First, we spread the zombies（僵尸开始移动啦！！！！！颤抖吧）
    """
    # Step 1: Zombies spread
    for zombies_cell_x in range(234):
        for zombies_cell_y in range(322):
            cell_start =  (zombies_cell_x, zombies_cell_y) #开始传播的点
            
            all_information_in_the_start_cell = all_information_in_each_cell[cell_start]  #当前点的所有信息
            if all_information_in_the_start_cell[2] > 0: # if there are zombies in this cell
            
            #  print(cell_start)
                #调用计算lamda的函数
                lamda_surround = calculate_lamda(cell_start)   
                #读取这个开始点的僵尸数量，为了计算传播到临近区域的僵尸数量
                amount_zombies_start = all_information_in_the_start_cell[2] 
                #总的僵尸数
                human_total_surround = the_total_human_amount_of_surround(cell_start)
            
                for degree_cell, lamda_value in lamda_surround.items():
                    #如果lamda的值大于0说明僵尸可以移动到该区域，给该区域附上信息
                    if lamda_value > 0:
#                       zombies_moving_dict, zombies_live_dict_moving, zombies_dict = zombies_spread(zombies_cell_x, zombies_cell_y, amount_zombies_start, zombies_dict)
                        all_information_in_the_destination_cell = all_information_in_each_cell[degree_cell]

                        zombies_wait_to_enter = 0
#                         print("-------------------------------------------------------------------------------------------")
                        if all_information_in_the_destination_cell[3]>0:
                                
#                                 print("human in destination cell", all_information_in_the_destination_cell[3])
                            
                                amount_human_destination = all_information_in_the_destination_cell[3]
#                                zombie_entree_to_destination = formule_zombies_to_cell_n(cell_start, cell_aim, lamda, human_total_surround, amount_human_destination, amount_zombies_start)
                                pourcentage_human = amount_human_destination/human_total_surround
                                cell_destination_zombies_entree = pourcentage_human*amount_zombies_start*lamda_value
                                
                                # Step 1.1: add information in temporary dictionary
                                zombies_wait_to_enter = cell_destination_zombies_entree
#                                 print("zombies_dict[cell_start]",zombies_dict[cell_start])
                                
                                for zombie_surviving_days in  range(15):
#                                     print("zombie_surviving_days",zombie_surviving_days)
                                    zombie_quantity = zombies_dict[cell_start][zombie_surviving_days]
#                                     print("zombie_quantity", zombie_quantity, "zombies_wait_to_enter",zombies_wait_to_enter)
#                                    for zombie_quantity in range(len(zombie_quantity_list)):
                                    if (zombie_quantity>=zombies_wait_to_enter) and (zombies_wait_to_enter>0):
                                        add_zombies=0
                                        del_zombies=0
#                                         print(add_surviving_info_temp_dict[degree_cell])
#                                         print("add_surviving_info_temp_dict",add_surviving_info_temp_dict[degree_cell][zombie_surviving_days])
                                        add_zombies = zombies_wait_to_enter + add_surviving_info_temp_dict[degree_cell][zombie_surviving_days]
                                        del_zombies = del_surviving_info_temp_dict[cell_start][zombie_surviving_days] + zombies_wait_to_enter
#                                         print("add_zombies",add_zombies,"del_zombies",del_zombies)
                                        add_surviving_info_temp_dict[degree_cell][zombie_surviving_days] = add_zombies
                                        del_surviving_info_temp_dict[cell_start][zombie_surviving_days] = del_zombies
                                        zombies_wait_to_enter = 0
                                    elif (zombie_quantity<zombies_wait_to_enter) and (zombies_wait_to_enter>0):
                                        add_zombies=0
                                        del_zombies=0
#                                         print("61: add_zombies",add_zombies,"del_zombies",del_zombies)
                                        add_zombies = zombie_quantity+add_surviving_info_temp_dict[degree_cell][zombie_surviving_days]
                                        del_zombies = del_surviving_info_temp_dict[cell_start][zombie_surviving_days] + zombie_quantity
                                        add_surviving_info_temp_dict[degree_cell][zombie_surviving_days]= add_zombies
                                        del_surviving_info_temp_dict[cell_start][zombie_surviving_days] = del_zombies
                                        zombies_wait_to_enter = zombies_wait_to_enter - zombie_quantity
                                    else:  #(zombie_quantity 0 zombies_wait_to_enter 0)
                                        break
#                                     print("***************************************************************")
                
                
                
                
                
    # Step 1.2: modify information in zombies_dict 
    for cell_in_temp_dict, value_in_temp_dict in add_surviving_info_temp_dict.items():
        for surviving_day, number_zombie in value_in_temp_dict.items():
            zombies_dict[cell_in_temp_dict][surviving_day] = zombies_dict[cell_in_temp_dict][surviving_day]+number_zombie
#             if(zombies_dict[cell_in_temp_dict][surviving_day]!=0):
#                 print("add_surviving_info_temp_dict",cell_in_temp_dict,  zombies_dict[cell_in_temp_dict])
    for cell_in_temp_dict, value_in_temp_dict in del_surviving_info_temp_dict.items():
        for surviving_day, number_zombie in value_in_temp_dict.items():
            zombies_dict[cell_in_temp_dict][surviving_day] = zombies_dict[cell_in_temp_dict][surviving_day]-number_zombie
#             if(zombies_dict[cell_in_temp_dict][surviving_day]!=0):
#                 print("del_surviving_info_temp_dict", cell_in_temp_dict, zombies_dict[cell_in_temp_dict])

    
    # Step 1.3: write zombie and human information to zombie_human
    for temp_cell_x in range(234):
        for temp_cell_y in range(322):
            zombie_human[(temp_cell_x,temp_cell_y)] = [zombies_dict[(temp_cell_x,temp_cell_y)], all_information_in_each_cell[(temp_cell_x,temp_cell_y)][3]]
#             if(zombie_human[(temp_cell_x,temp_cell_y)]!=[0,0]):
#                 print("zombie_human",(temp_cell_x,temp_cell_y),  zombie_human[(temp_cell_x,temp_cell_y)][0])

            
            
            
    """
    After spreading, we begin to fight 
    """
            
    for zombie_human_cell, zombie_and_human in zombie_human.items():
#             print(zombie_human)
#             print(zombie_human_cell)
#             print(zombie_and_human)


            # Step 2: Zombies kill people
#             after_zkp, zombies_dict = zombie_kill_human(zkp_cell_x, , zombie_human[zkp_cell_x, zkp_cell_y], zombies_dict)
        if (sum(zombie_and_human[0])!=0 and zombie_and_human[1]!=0):
            the_zombie_inside_cell = sum(zombie_and_human[0])
            the_human_inside_cell = zombie_and_human[1]

            human_after_killing = the_human_inside_cell - 10*the_zombie_inside_cell
            if (human_after_killing < 0):#the cell is destroyed;no more human 如果人类的数量小于十倍的僵尸数量，该区域的人将会全部变成僵尸
                zombies_dict[zombie_human_cell][0] = the_human_inside_cell
                the_zombie_inside_cell = the_zombie_inside_cell+the_human_inside_cell #僵尸进攻完后的数量；人会变成僵尸
                the_human_inside_cell = 0
                
            else:#human will revenge；如果人类的数量大于十倍的僵尸数量，人类有剩余，在第三步将进行进攻
                zombies_dict[zombie_human_cell][0] = 10*the_zombie_inside_cell + zombies_dict[zombie_human_cell][0]
                the_zombie_inside_cell = 11*the_zombie_inside_cell
                the_human_inside_cell = human_after_killing

    
    
            # Step 3: People kill Zombies
#             after_pkz = human_kill_zombie(zkp_cell_x,zkp_cell_y, after_zkp)

            if (the_human_inside_cell!=0):#如果第二步进行完后人类数量不为0，说明该区剩余的人将会反击,（if the human）
                zombie_after_killed=the_zombie_inside_cell-10*the_human_inside_cell
                
                
                if (zombie_after_killed<0): #说明人类反击成功，该区僵尸被消灭，人类有剩余
                    the_human_inside_cell = the_human_inside_cell
                    the_zombie_inside_cell = 0
                    for surviving_days in range(15):
#                         amount = zombies_dict[zombie_human_cell][surviving_days]
                        zombies_dict[zombie_human_cell][surviving_days] = 0

                else :#僵尸的数量要比10倍人类要多，人类会消灭一部分的僵尸，这些人类会活到第二天；第二天僵尸可能会移动，也可能会继续留在该cell（如果没有别的区域可以扩散）
                    the_human_inside_cell = the_human_inside_cell
                    the_zombie_inside_cell = zombie_after_killed
                    zombies_going_to_be_killed = 10*the_human_inside_cell
                    
                    for surviving_days in range(15):
                        print("zombies_dict[zombie_human_cell]",zombie_human_cell,surviving_days,zombies_dict[zombie_human_cell])
                        print(zombies_dict[zombie_human_cell][surviving_days])
                        amount_zombie_in_certain_days = zombies_dict[zombie_human_cell][surviving_days]
                        print("amount_zombie_in_certain_days",amount_zombie_in_certain_days)
                        print("zombies_going_to_be_killed",zombies_going_to_be_killed)
                        print("-------------------------------------------------------")
                        
                        if(amount_zombie_in_certain_days >= zombies_going_to_be_killed) and (zombies_going_to_be_killed>0):
                            last_zombies = amount_zombie_in_certain_days-zombies_going_to_be_killed
                            zombies_dict[zombie_human_cell][surviving_days]=last_zombies
                            zombies_going_to_be_killed = 0
                            
                        elif (amount_zombie_in_certain_days < zombies_going_to_be_killed) and (zombies_going_to_be_killed>0):
                            last_zombies = 0
                            zombies_going_to_be_killed = zombies_going_to_be_killed-amount_zombie_in_certain_days
                            zombies_dict[zombie_human_cell][surviving_days]=last_zombies
                        
                        elif zombies_going_to_be_killed == 0:
                            break
                            
                        else:
                            break
                        

            zombie_and_human[0] = zombies_dict[zombie_human_cell]
            zombie_and_human[1] = the_human_inside_cell
            
            zombie_human[zombie_human_cell] = zombie_and_human
            
            

    """
    After one day, we begin to updateinformation 
    """
            
    # Step 4: Update zombies and human infomation into all_information_in_each_cell
    for cell, zom_hu in zombie_human.items():
#         print(int(sum(zom_hu[0])))
        amount_of_zombies = int(sum(zom_hu[0]))
#         print(type(amount_of_zombies))
        amount_of_humans = zom_hu[1]
        all_information_in_each_cell[cell][2] = amount_of_zombies
        all_information_in_each_cell[cell][3] = amount_of_humans
            
            
            
    # Step 5: Update zombies surviving days
    for cell, survivng_list in zombies_dict.items():
        old_surviving_list = survivng_list
        old_surviving_list.insert(0,0)
#         if(sum(old_surviving_list)!=0):
#             print(old_surviving_list)
        if len(old_surviving_list)==16:
            del old_surviving_list[-1]
        zombies_dict[cell] = old_surviving_list
    
#     print("")
#     print("********************")
#     print("FINISH")
#     print("********************")
#     print("")
        

    return (all_information_in_each_cell, zombies_dict)
        


            
                
                

In [49]:
while(all_information_in_each_cell[86,43][2]==0):
    day = day+1
    print(day)
    zombies_evolution(all_information_in_each_cell, zombies_dict)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
zombies_dict[zombie_human_cell] (120, 289) 0 [3868.7273509707916, 386.87273509707916, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3868.7273509707916
amount_zombie_in_certain_days 3868.7273509707916
zombies_going_to_be_killed 1622.7264902920842
-------------------------------------------------------
zombies_dict[zombie_human_cell] (120, 289) 1 [2246.0008606787073, 386.87273509707916, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
386.87273509707916
amount_zombie_in_certain_days 386.87273509707916
zombies_going_to_be_killed 0
-------------------------------------------------------
24
25
26
27
zombies_dict[zombie_human_cell] (118, 319) 0 [10701.538710897401, 1070.15387108974, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10701.538710897401
amount_zombie_in_certain_days 10701.538710897401
zombies_going_to_be_killed 4544.61289102599
-------------------------------------------------------
zombies_dict[zombie_human_cell] (118, 319) 1 [6156.925819871411, 1

94
95
96
97
98
99
100
101
zombies_dict[zombie_human_cell] (63, 212) 0 [62649.10998641291, 6264.9109986412905, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
62649.10998641291
amount_zombie_in_certain_days 62649.10998641291
zombies_going_to_be_killed 41708.900135870936
-------------------------------------------------------
zombies_dict[zombie_human_cell] (63, 212) 1 [20940.20985054197, 6264.9109986412905, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6264.9109986412905
amount_zombie_in_certain_days 6264.9109986412905
zombies_going_to_be_killed 0
-------------------------------------------------------
zombies_dict[zombie_human_cell] (63, 215) 0 [7563.263394019782, 756.3263394019782, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7563.263394019782
amount_zombie_in_certain_days 7563.263394019782
zombies_going_to_be_killed 1237.3660598021797
-------------------------------------------------------
zombies_dict[zombie_human_cell] (63, 215) 1 [6325.897334217602, 756.3263394019782, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
zombies_dict[zombie_human_cell] (56, 143) 0 [102567.08966851106, 10256.708966851107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
102567.08966851106
amount_zombie_in_certain_days 102567.08966851106
zombies_going_to_be_killed 112219.10331488936
-------------------------------------------------------
zombies_dict[zombie_human_cell] (56, 143) 1 [0, 10256.708966851107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10256.708966851107
amount_zombie_in_certain_days 10256.708966851107
zombies_going_to_be_killed 9652.013646378298
-------------------------------------------------------
zombies_dict[zombie_human_cell] (56, 143) 2 [0, 604.6953204728088, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0
amount_zombie_in_certain_days 0
zombies_going_to_be_killed 0
-------------------------------------------------------
155
156
157
158
159
160
161
162
163
164
165
166
zombies_dict[zombie_human_cell] (45, 131) 0 [133980.29993202002, 13398.029993202, 0, 

In [31]:
day

253